In [2]:
import pybliometrics.scopus as scopus
import numpy as np
import pandas as pd
import random
import time

In [3]:
seed = 7 # seed for NMF topic model
num_topics = 12
query = 'title-abs-key("engineering" AND "research" AND "innovation") AND PUBYEAR = 2019'
labels = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
vis_seed = 6 # seed for t-SNE visualization
vis_angle = 135 # rotation angle for visualization

In [4]:
scopus.init()
papers = scopus.ScopusSearch(query, subscriber=True, verbose=True)
print(papers.results)

[Document(eid='2-s2.0-85159137407', doi='10.11959/j.issn.2096-3750.2019.00139', pii=None, pubmed_id=None, title='Research on engineering parameter perception and innovation application based on Internet of things', subtype='ar', subtypeDescription='Article', creator='Li Y.', afid='60001081', affilname='China Telecom Corporation Limited', affiliation_city='Beijing', affiliation_country='China', author_count='2', author_names='Li, Yanfen;Zhu, Xuetian', author_ids='58245566900;57211986298', author_afids='60001081;60001081', coverDate='2019-12-30', coverDisplayDate='30 December 2019', publicationName='Chinese Journal on Internet of Things', issn='20963750', source_id='21101052723', eIssn=None, aggregationType='Journal', volume='3', issueIdentifier='4', article_number=None, pageRange='116-120', description='Antenna engineering parameters are the benchmark of the wireless network planning and optimization. It is of great significance for reducing the difficulty of the network operation and o

In [4]:
df = pd.DataFrame(papers.results)

In [5]:
# สร้าง DataFrame เพื่อบันทึกเป็น CSV
data = []

for paper in papers.results:
    # Format the author names
    if hasattr(paper, 'author_names') and paper.author_names:
        # Split the author names by ';' and remove leading/trailing spaces
        authors = paper.author_names.split(';')
        authors = [author.strip() for author in authors]  # Strip extra spaces

        formatted_authors = []
        for author in authors:
            name_parts = author.split(',')
            if len(name_parts) == 2:  # If the name has both Lastname and Firstname
                last_name = name_parts[0].strip()
                first_name = name_parts[1].strip()
                # Only append the initial if there is a first name
                if first_name:
                    formatted_authors.append(f"{last_name} {first_name[0]}.")
                else:
                    formatted_authors.append(f"{last_name}.")  # In case of missing first name
            else:  # If the name is a single part (like just Lastname)
                formatted_authors.append(f"{name_parts[0].strip()}.")

        # Join formatted authors with "; "
        formatted_authors = "; ".join(formatted_authors)
    else:
        formatted_authors = ""

    paper_dict = {
        "Title": paper.title,
        "Abstract": paper.description if hasattr(paper, 'description') else "",
        "Author": formatted_authors,
        "Aggregation_Type": paper.aggregationType if hasattr(paper, 'aggregationType') else "",
        "Publisher": paper.publicationName if hasattr(paper, 'publicationName') else "",
        "Publication_Date": paper.coverDate if hasattr(paper, 'coverDate') else "",
        "Institutions": "; ".join(paper.affilname.split('; ')) if hasattr(paper, 'affilname') and paper.affilname else "",
        "Keywords": "; ".join(paper.authkeywords.split('|')) if hasattr(paper, 'authkeywords') and paper.authkeywords else "",
    }
    data.append(paper_dict)



df = pd.DataFrame(data)

output_file = "scopus_papers_2019_full_details.csv"
df.to_csv(output_file, index=False, encoding="utf-8")